In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import efficientnet
import efficientnet.tfkeras as efn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.layers import GlobalAveragePooling2D
# import tensorflow_addons as tfkeras
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2 as cv                 # opencv (이미지 resize)
import easydict

In [ ]:
train_dir = './image_data_CNN'


# ImageDataGenerator를 생성
# RGB 값(0~255로 구성)으로 각 픽셀의 값을 나눠서 정규화 (MinMaxScaling)
train_datagen = ImageDataGenerator(rescale=1/255,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,    # target directory
    classes=['disable1', 'disable2', 'disable3'], # 폴더명을 class로 잡는다.
                              # 따로 명시하지 않으면 오름차순으로 명시  
    target_size=(150,150),
    batch_size=8,
    class_mode='sparse',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,                 # target directory
    classes=['disable1', 'disable2', 'disable3'], # 폴더명을 class로 잡는다.
    target_size=(150,150),
    batch_size=8,
    class_mode='sparse',
    subset='validation')

In [ ]:
# pretrained network

pretrained_network = efn.EfficientNetB4(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3))

print(pretrained_network.summary())

pretrained_network.trainable = False

# keras 구현
# model 구현

model = Sequential()

model.add(pretrained_network)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=3,
                activation='softmax'))

In [ ]:
# Early Stopping
es = EarlyStopping(monitor='val_loss',
                   mode='auto',
                   patience=5,
                   verbose=1)

model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator)//8,
                    steps_per_epoch=len(train_generator)//8,
                    epochs=300,
                    verbose=1)

In [ ]:
print(type(history))
print(history.history.keys())

plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.show()

In [ ]:
print(type(history))
print(history.history.keys())

plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'], color='b')
plt.show()

In [ ]:
model.save('./e_pretrain_5.h5')

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model('./e_pretrain_5.h5')

In [ ]:
args = easydict.EasyDict(
    {'exp_num':'0',
     'Kfold':5,
    })

In [ ]:
pretrained_network.trainable =  True    # Convolution Layer안의 parameter 동결 해제

for layer in pretrained_network.layers:
    if layer.name in ['block7b_se_reduce', 'block7b_se_expand', 'block7b_project_conv', 'top_conv']:
        layer.trainable = True  # 동결 해제
    else:
        layer.trainable = False # 동결

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

for s_fold in range(5): # 5fold
    args.fold = s_fold
    args.exp_num = str(s_fold)
    history = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator)//8,
                    steps_per_epoch=len(train_generator)//8,
                    epochs=300,
                    verbose=1)
    print(type(history))
    print(history.history.keys())
    plt.plot(history.history['accuracy'], color='r')
    plt.plot(history.history['val_accuracy'], color='b')
    plt.show()
    
    print(type(history))
    print(history.history.keys())
    plt.plot(history.history['loss'], color='r')
    plt.plot(history.history['val_loss'], color='b')
    plt.show()


In [ ]:
model.save('./final_model_300.h5')